# Mean Squared Error vs Cross Entropy 

In [2]:
import numpy as np

In [8]:
def mean_squared_error(y, p):
    return ((y - p)**2).mean()

def cross_entropy(y, p):
    return -(y * np.log(p)).sum()

### Mean Squared Error를 사용해야하는 곳

one-hot vector를 이용한 classification사용시 MSE를 사용해야 합니다.

In [13]:
y_pred = np.array([0.1, 0.20, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.5])
y_true = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

print('cross entropy     :\t', cross_entropy(y_true, y_pred))
print('mean squared error:\t', mean_squared_error(y_true, y_pred))

cross entropy     :	 2.30258509299
mean squared error:	 0.39
